In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from analysis.processor import Processor

In [13]:
from analysis.selutility import Object

In [15]:
from config.selectionconfig import settings as sel_cfg

In [5]:
from analysis.helper import *

In [6]:
from config.selectionconfig import runsetting as rs

In [7]:
import random
import json
import os

In [8]:
# custom-define datasets to process 
process_names = ["TTbar", "ggF", "SingleH"]
data_src = 'src/data/preprocessed'

In [10]:
# Quick hack
ds_names = ["DYJets", "TTbar", "ggF"]
filelist = ["root://cmsxrootd.fnal.gov//store/mc/Run3Winter22NanoAOD/DYJetsToLL_M-10to50_TuneCP5_13p6TeV-madgraphMLM-pythia8/NANOAODSIM/122X_mcRun3_2021_realistic_v9-v1/30000/10e16df1-d8bb-4b32-bb90-32c4fd57f429.root",
           "root://cmsxrootd.fnal.gov//store/mc/Run3Winter22NanoAOD/TTToSemiLeptonic_TuneCP5_13p6TeV-powheg-pythia8/NANOAODSIM/FlatPU0to70_pilot_122X_mcRun3_2021_realistic_v9-v1/30000/1fa8ebbc-f09a-4244-b41f-5125c5ed9f48.root",
           "root://cmsxrootd.fnal.gov//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/5998ad11-ab82-41bf-a740-7a2c38b85a14.root"]

In [11]:
finame = filelist[2]
proc = Processor(rs, 'ggF')
events = proc.loadfile(finame, 'test')

In [14]:
# Create test object: Tau
objcfg = sel_cfg.signal.outputs.Tau
tau = Object("Tau", events, sel_cfg.signal.outputs.Tau, )

AttributeError: 'Processor' object has no attribute 'output_cfg'

In [9]:
for i, ds in enumerate(ds_names):
    proc = Processor(rs, ds)
    proc.runfile(filelist[i], 0, None)

In [15]:
output_fi = os.listdir(rs.OUTPUTDIR_PATH)

In [21]:
dfs = []
for file_name in output_fi:
    df = pd.read_csv(pjoin(rs.OUTPUTDIR_PATH, file_name), index_col=0, header=0)
    df.columns = [file_name]
    dfs.append(df)

In [22]:
raw_df = pd.concat(dfs, axis=1)

In [23]:
raw_df

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,506334,927960,80104
ElectronSelection,500522,525938,45205
MuonSelection,488033,289438,31124
TauSelection,361,32286,14806


In [24]:
efficiency_df = raw_df.div(raw_df.shift(1)).fillna(1)

In [25]:
# incremental
efficiency_df

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,1.000000,1.000000,1.000000
ElectronSelection,0.988521,0.566768,0.564329
MuonSelection,0.975048,0.550327,0.688508
TauSelection,0.000740,0.111547,0.475710


In [26]:
overall_eff = raw_df.div(raw_df.iloc[0]).fillna(1)

In [27]:
# wrt to total
overall_eff

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,1.000000,1.000000,1.000000
ElectronSelection,0.988521,0.566768,0.564329
MuonSelection,0.963856,0.311908,0.388545
TauSelection,0.000713,0.034792,0.184835


In [ ]:
# Run small scale selections on one file for each process
def runsample(dsnames, datasrc):
    samplefl = []
    for name in dsnames:
        with open(pjoin(datasrc, f'{name}.json'), 'r') as file:
            meta = json.load(file)
            samplefl.append(meta)